In [19]:
import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import seaborn as sns

In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [22]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

In [23]:
import PPIGraph

# Read per cell data

In [24]:
# Read PPI
PPI_save_path =  data_dir / '13cyc' / 'PPI'

dfs = []
for path in os.listdir(PPI_save_path):
    if 'csv' in path:
        df = pd.read_csv(PPI_save_path / path)
        dfs.append(df)

df = pd.concat(dfs)

# Group by location (Cell vs Nuclei)
g = df.groupby(['Condition', 'FOV', 'PPI', 'Cyto']).size()
df_cell = pd.DataFrame({'Count Cyto': g}).reset_index()
df_cell = df_cell[df_cell.Cyto != 0]
df_cell.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Count_cell']

g = df.groupby(['Condition', 'FOV', 'PPI', 'Nuclei']).size()
df_nuclei = pd.DataFrame({'Count Nuclei': g}).reset_index()
df_nuclei= df_nuclei[df_nuclei.Nuclei != 0]
df_nuclei.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Count_nuclei']

# Concat 
df_all = df_cell.merge(df_nuclei, how='left', on=['Condition', 'FOV', 'PPI', 'Id']).fillna(0)
df_all['Count_nuclei'] = df_all['Count_nuclei'].astype(int)
df_all['Count_cyto'] = df_all['Count_cell'] - df_all['Count_nuclei'] # Cyto count

# Ranme columns
df_all.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Cell', 'Nuclei',
       'Cyto']

# Plot descrition
display(df_all.describe())
display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())

,Id,Cell,Nuclei,Cyto
count,24782.000000,24782.000000,24782.000000,24782.000000
mean,451.891453,8.109111,2.371883,5.737229
std,264.699437,10.532796,4.842329,9.876917
min,21.000000,1.000000,0.000000,-157.000000
25%,227.000000,2.000000,0.000000,1.000000
50%,431.500000,5.000000,1.000000,3.000000
75%,649.000000,9.000000,3.000000,7.000000
max,1071.000000,232.000000,205.000000,232.000000


C:\Users\thu71\AppData\Local\Temp\ipykernel_57960\3196296033.py:34: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())


,Cell,Nuclei,Cyto
count,2236.000000,2236.000000,2236.000000
mean,89.874776,26.288014,63.586762
std,49.101315,27.687732,53.779516
min,1.000000,0.000000,-434.000000
25%,59.000000,0.000000,29.000000
50%,81.000000,25.000000,56.000000
75%,109.000000,41.000000,89.000000
max,748.000000,577.000000,748.000000


In [25]:
# Filter out by maximum number of counts per cell
min_count = 0
max_count = 50

df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: x['Cell'].sum() > min_count)
df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: (x['Cell'] < max_count).all())
# df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: (x['Cyto'] >= 0).all())

# Plot descrition
display(df_all.describe())
display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())

,Id,Cell,Nuclei,Cyto
count,21982.000000,21982.000000,21982.000000,21982.000000
mean,451.329133,7.261441,2.386680,4.874761
std,263.249013,8.066408,4.827968,7.495634
min,21.000000,1.000000,0.000000,-157.000000
25%,231.000000,2.000000,0.000000,1.000000
50%,430.000000,4.000000,1.000000,3.000000
75%,649.000000,9.000000,3.000000,7.000000
max,1071.000000,49.000000,205.000000,49.000000


C:\Users\thu71\AppData\Local\Temp\ipykernel_57960\2030770136.py:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())


,Cell,Nuclei,Cyto
count,2007.000000,2007.000000,2007.000000
mean,79.532138,26.140508,53.391629
std,32.651668,27.234153,40.010580
min,1.000000,0.000000,-434.000000
25%,57.000000,0.000000,27.000000
50%,77.000000,25.000000,51.000000
75%,99.000000,41.000000,79.000000
max,208.000000,577.000000,194.000000


# ML model

In [31]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing, metrics
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, KFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler
import wandb

In [32]:
scaler = StandardScaler()

In [33]:
condition = 'ML'
project_name = f'PLA_13PPI_batch'

In [34]:
models = {
    'Adaboost': AdaBoostClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'GradientBoosting' : GradientBoostingClassifier(),
    'NaiveBayes': GaussianNB(),
    'RandomForest': RandomForestClassifier(), 
    'SVM': SVC(probability =True),
    'LogisticRegression':  LogisticRegression(),
    'MLP': MLPClassifier(random_state=1, max_iter=100, hidden_layer_sizes=[16, 16, 16])
}



In [35]:
# Get dataframe seperate into count per cell 
df_count = df_all.pivot(index=['Condition', 'FOV', 'Id'], columns=['PPI']).fillna(0)
df_count_cell = df_count.iloc[:, df_count.columns.get_level_values(0)=='Cell']
df_count_cell = pd.DataFrame(scaler.fit_transform(df_count_cell), index=df_count_cell.index, columns=df_count_cell.columns, )

# Get dataframe seperate into count per nuclei and cyto
df_count_cyto_nuclei = df_count.iloc[:, df_count.columns.get_level_values(0)!='Cell']

# Get Condition into numerical label
le = preprocessing.LabelEncoder()
y = le.fit_transform(df_count.index.get_level_values(0))

In [36]:
df_count.index.get_level_values('FOV').unique()

Index(['FW1', 'FW2'], dtype='object', name='FOV')

In [37]:
df_count_cell 

Cell                                                   \
PPI                AKT/Mtor Bim/Tom20 Cyclin D1/CDK4 Cyclin E/CDK2 EGFR/GRB2   
Condition FOV Id                                                               
100nM     FW1 36  -0.925167  0.257262       0.055826     -0.894986 -0.600407   
              37   1.999866 -0.566223       0.055826     -0.157312 -0.314790   
              41   0.829852 -0.154480      -0.172832      0.580363 -1.028832   
              47   0.244846 -0.977965      -0.858804      0.211526 -0.600407   
              53   0.829852 -0.154480      -0.858804      0.949200 -0.171982   
...                     ...       ...            ...           ...       ...   
control   FW2 599 -0.340161  0.669005      -0.172832      0.949200  1.827335   
              602  2.584872  1.492489       2.113742      0.580363  0.684868   
              605  0.244846 -0.977965      -0.401489     -0.894986 -0.743215   
              608  1.414859 -0.977965      -0.172832      2.793387 -0.029174   
              609  0.829852 -1.389708       0.970455      0.949200 -0.314790   

                                                                          \
PPI               FoxO1/AKT Mcl-1/BAK NF-Kb/p-P90rsk SIRT1/P53 Sox2/Oct4   
Condition FOV Id                                                           
100nM     FW1 36   0.505886  0.790900       0.071531 -0.722599 -0.891343   
              37  -0.788166 -0.190755      -0.216747 -0.722599 -0.172755   
              41  -0.788166  1.281727      -0.793303 -0.722599 -1.250637   
              47  -0.529356  0.790900      -0.793303 -0.722599 -0.891343   
              53   0.505886 -0.190755       0.071531 -0.722599  0.905128   
...                     ...       ...            ...       ...       ...   
control   FW2 599 -1.046977  0.300072       0.071531 -0.722599  3.060893   
              602  2.317560  2.263382       1.224642  0.422943  0.905128   
              605 -0.529356 -0.681582      -1.369858 -0.722599 -0.172755   
              608 -0.529356  0.790900       1.224642 -0.722599  0.186539   
              609  0.247076  1.281727      -0.505025 -0.722599 -0.172755   

                                                   
PPI               TEAD1/YAP TRAIL/DR5 p-ERK/c-MYC  
Condition FOV Id                                   
100nM     FW1 36  -0.447144 -0.683428    0.251815  
              37  -0.975037 -0.781655    0.597560  
              41  -0.447144 -0.978108    2.844901  
              47  -0.975037 -0.585201   -0.180366  
              53   0.080749  0.298841    1.548358  
...                     ...       ...         ...  
control   FW2 599  0.080749  0.986428    1.980539  
              602  2.192321  1.084655    2.758465  
              605  0.608642 -1.469242   -0.439675  
              608  1.664428  0.789975    0.770432  
              609  2.720214  0.789975   -0.439675  

[2007 rows x 13 columns]

In [40]:
scaler = StandardScaler()

In [41]:
## Train on P10 an'P10d evaluate on P21
train = 'FW1'
held = 'FW2'

# Run model on cell count
df_train = df_count_cell[df_count.index.get_level_values('FOV') == train]
df_held = df_count_cell[df_count.index.get_level_values('FOV') == held]
X = df_train.values
y = le.transform(df_train.index.get_level_values(0))
X_held = df_held.values
y_held = le.transform(df_held.index.get_level_values(0))

print(len(X), len(y), len(X_held), len(y_held))
print(np.unique(y, return_counts=True), np.unique(y_held, return_counts=True))

1231 1231 776 776
(array([0, 1]), array([531, 700], dtype=int64)) (array([0, 1]), array([447, 329], dtype=int64))


In [42]:
## Train on P10 an'P10d evaluate on P21
train = 'FW1'
held = 'FW2'

# Run model on cell count
df_train = df_count_cell[df_count_cell.index.get_level_values('FOV') == train]
df_held = df_count_cell[df_count_cell.index.get_level_values('FOV') == held]

X = df_train.values
y = le.transform(df_train.index.get_level_values(0))

X_held = df_held.values
y_held = le.transform(df_held.index.get_level_values(0))

print(len(X), len(y), len(X_held), len(y_held))

# K fold training
kfold = KFold(n_splits = 3, shuffle = True, random_state = 0)
for model_name, model in models.items():
    for k, (train_index, test_index) in enumerate(kfold.split(X)):
        # Split the dataset
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index] 

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probas = model.predict_proba(X_test)
        y_pred_held = model.predict(X_held)
        y_probas_held = model.predict_proba(X_held)

        run = wandb.init(project=project_name, group=model_name+'_'+train, name=model_name+f'_{train}_{k}')

        accuracy = metrics.accuracy_score(y_test, y_pred)
        accuracy_held = metrics.accuracy_score(y_held, y_pred_held)
        f1 = metrics.f1_score(y_test, y_pred)
        f1_held = metrics.f1_score(y_held, y_pred_held)
        # auc = metrics.roc_auc_score(y_test, y_probas[:, 1])
        # auc_held = metrics.roc_auc_score(y_held, y_probas_held[:, 1])
        auc = metrics.roc_auc_score(y_test, y_pred)
        auc_held = metrics.roc_auc_score(y_held, y_pred_held)
        wandb.log({"accuracy": accuracy, 'accuracy_held': accuracy_held, 'f1':f1, 'f1_held': f1_held, 'auc': auc, 'auc_held': auc_held})
    run.finish()

1231 1231 776 776


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thoomas. Use `wandb login --relogin` to force relogin


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.90754
accuracy_held,0.61727
auc,0.90906
auc_held,0.56147
f1,0.91915


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.85854
accuracy_held,0.61856
auc,0.85808
auc_held,0.56821
f1,0.86449


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.88293
accuracy_held,0.60438
auc,0.87879
auc_held,0.54748
f1,0.904


...

c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.85401
accuracy_held,0.625
auc,0.85567
auc_held,0.56818
f1,0.87124


c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.84146
accuracy_held,0.63144
auc,0.84004
auc_held,0.58902
f1,0.85714


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.88049
accuracy_held,0.6366
auc,0.87445
auc_held,0.58908
f1,0.90258
